In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Atherosclerosis"
cohort = "GSE83500"

# Input paths
in_trait_dir = "../../input/GEO/Atherosclerosis"
in_cohort_dir = "../../input/GEO/Atherosclerosis/GSE83500"

# Output paths
out_data_file = "../../output/preprocess/Atherosclerosis/GSE83500.csv"
out_gene_data_file = "../../output/preprocess/Atherosclerosis/gene_data/GSE83500.csv"
out_clinical_data_file = "../../output/preprocess/Atherosclerosis/clinical_data/GSE83500.csv"
json_path = "../../output/preprocess/Atherosclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from aortic wall between myocardial infarction (MI) and non-MI group"
!Series_summary	"The aortic wall of patients with ischemic heart disease may have an indicative characteristic of mRNA predictive of future cardiovascular events."
!Series_summary	"We used microarrays to detail the gene expression and identified distinct classes of up-regulated and down-regulated genes."
!Series_overall_design	"Ascending aortic wall punch biopsies obtained as a standard part of coronary artery bypass surgery, will be used as a novel approach to study the vessel wall of patients with atherosclerosis. A total of 37 (17 MI, 20 Non-MI) frozen aortic tissues  were embedded in TissueTek optimal cutting temperature (OCT) compound (TissueTek; Sakura Finetek USA). The embedded aortic tissues were trimmed and sectioned to a thickness of 10µm and placed on an RNase-free Polyethylene Naphthalate (PEN) membrane slide (Carl Zeiss; Germany). Haematoxylin and eo

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, the dataset contains gene expression data from aortic tissue
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait, row 0 has "individual: MI patient" vs "individual: non-MI patient"
trait_row = 0

# For age, row 1 has "age: XX" values
age_row = 1

# For gender, row 2 has "Sex: Male" vs "Sex: Female"
gender_row = 2

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert MI/non-MI to binary values (1/0)"""
    if value is None:
        return None
    value = value.lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'mi patient' in value:
        return 1  # MI patient
    elif 'non-mi patient' in value:
        return 0  # non-MI patient
    return None

def convert_age(value):
    """Convert age string to numeric value"""
    if value is None:
        return None
    if ':' in value:
        try:
            return int(value.split(':', 1)[1].strip())
        except (ValueError, TypeError):
            return None
    return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    value = value.lower()
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    return None

# 3. Save Metadata - Initial filtering
# Trait data is available if trait_row is not None
is_trait_available = trait_row is not None

# Validate and save cohort info (initial filtering)
is_usable = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since this is the analysis step and we don't yet have clinical_data.csv, 
# we're just identifying available variables and their conversion methods.
# The actual clinical data loading and processing will happen in subsequent steps.
print(f"Gene Expression Available: {is_gene_available}")
print(f"Trait Available: {is_trait_available}")
print(f"Trait Row: {trait_row}")
print(f"Age Row: {age_row}")
print(f"Gender Row: {gender_row}")


Gene Expression Available: True
Trait Available: True
Trait Row: 0
Age Row: 1
Gender Row: 2


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Atherosclerosis/GSE83500/GSE83500_series_matrix.txt.gz


Gene data shape: (49386, 37)
First 20 gene/probe identifiers:
Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers shown (11715100_at, etc.) are probe IDs from a microarray platform,
# not standard human gene symbols. These will need to be mapped to gene symbols.
# Affymetrix probe IDs typically follow this pattern with "_at" suffixes.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's explore the SOFT file more thoroughly to find gene symbols
print("\nExploring SOFT file more thoroughly for gene information:")
gene_info_patterns = []
entrez_to_symbol = {}

with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if i < 1000:  # Check header section for platform info
            if '!Series_platform_id' in line or '!Platform_title' in line:
                print(line.strip())
                
        # Look for gene-related columns and patterns in the file
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line or 'Symbol' in line:
            gene_info_patterns.append(line.strip())
            
        # Extract a mapping using ENTREZ_GENE_ID if available
        if len(gene_info_patterns) < 2 and 'ENTREZ_GENE_ID' in line and '\t' in line:
            parts = line.strip().split('\t')
            if len(parts) >= 2:
                try:
                    # Attempt to add to mapping - assuming ENTREZ_GENE_ID could help with lookup
                    entrez_id = parts[1]
                    probe_id = parts[0]
                    if entrez_id.isdigit() and entrez_id != probe_id:
                        entrez_to_symbol[probe_id] = entrez_id
                except:
                    pass
        
        if i > 10000 and len(gene_info_patterns) > 0:  # Limit search but ensure we found something
            break

# Show some of the patterns found
if gene_info_patterns:
    print("\nFound gene-related patterns:")
    for pattern in gene_info_patterns[:5]:
        print(pattern)
else:
    print("\nNo explicit gene info patterns found")

# Let's try to match the ENTREZ_GENE_ID to the probe IDs
print("\nAnalyzing ENTREZ_GENE_ID column:")
if 'ENTREZ_GENE_ID' in gene_annotation.columns:
    # Check if ENTREZ_GENE_ID contains actual Entrez IDs (different from probe IDs)
    gene_annotation['ENTREZ_GENE_ID'] = gene_annotation['ENTREZ_GENE_ID'].astype(str)
    different_ids = (gene_annotation['ENTREZ_GENE_ID'] != gene_annotation['ID']).sum()
    print(f"Number of entries where ENTREZ_GENE_ID differs from ID: {different_ids}")
    
    if different_ids > 0:
        print("Some ENTREZ_GENE_ID values differ from probe IDs - this could be useful for mapping")
        # Show examples of differing values
        diff_examples = gene_annotation[gene_annotation['ENTREZ_GENE_ID'] != gene_annotation['ID']].head(5)
        print(diff_examples)
    else:
        print("ENTREZ_GENE_ID appears to be identical to probe ID - not useful for mapping")

# Search for additional annotation information in the dataset
print("\nLooking for alternative annotation approaches:")
print("- Checking for platform ID or accession number in SOFT file")

platform_id = None
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Platform_geo_accession' in line:
            platform_id = line.split('=')[1].strip().strip('"')
            print(f"Found platform GEO accession: {platform_id}")
            break
        if i > 200:
            break

# If we don't find proper gene symbol mappings, prepare to use the ENTREZ_GENE_ID as is
if 'ENTREZ_GENE_ID' in gene_annotation.columns:
    print("\nPreparing provisional gene mapping using ENTREZ_GENE_ID:")
    mapping_data = gene_annotation[['ID', 'ENTREZ_GENE_ID']].copy()
    mapping_data.rename(columns={'ENTREZ_GENE_ID': 'Gene'}, inplace=True)
    print(f"Provisional mapping data shape: {mapping_data.shape}")
    print(preview_df(mapping_data, n=5))
else:
    print("\nWarning: No suitable mapping column found for gene symbols")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GeneChip Array', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Transcript ID(Array Design)', 'Target Description', 'Representative Public ID', 'Archival UniGene Cluster', 'UniGene ID', 'Genome Version', 'Alignments', 'Gene Title', 'Gene Symbol', 'Chromosomal Location', 'GB_LIST', 'SPOT_ID', 'Unigene Cluster Type', 'Ensembl', 'Entrez Gene', 'SwissProt', 'EC', 'OMIM', 'RefSeq Protein ID', 'RefSeq Transcript ID', 'FlyBase', 'AGI', 'WormBase', 'MGI Name', 'RGD Name', 'SGD accession number', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function', 'Pathway', 'InterPro', 'Trans Membrane', 'QTL', 'Annotation Description', 'Annotation Transcript Cluster', 'Transcript Assignments', 'Annotation Notes']
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome HG-U219 Array', 'Human 

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify columns containing probe IDs and gene symbols in the gene annotation dataframe
# Based on previous output, we can see:
# - 'ID' column contains probe IDs (matching gene_data.index format like "11715100_at")
# - 'Gene Symbol' column contains gene symbols (like "HIST1H3G", "TNFAIP8L1")

# 2. Create gene mapping dataframe
mapping_data = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')
print(f"Gene mapping dataframe shape: {mapping_data.shape}")
print("\nGene mapping sample:")
print(preview_df(mapping_data, n=5))

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)
print(f"\nGene expression data shape after mapping: {gene_data.shape}")
print("\nFirst 10 genes after mapping:")
print(gene_data.index[:10].tolist())

# Check if genes with multiple probes were properly aggregated
gene_counts = mapping_data['Gene'].value_counts()
multi_probe_genes = gene_counts[gene_counts > 1]
print(f"\nNumber of genes mapped from multiple probes: {len(multi_probe_genes)}")
if len(multi_probe_genes) > 0:
    print("Top 5 genes with the most probes:")
    print(multi_probe_genes.head())

# Normalize gene symbols to ensure consistency (handle synonyms)
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nGene data shape after normalization: {gene_data.shape}")


Gene mapping dataframe shape: (49384, 2)

Gene mapping sample:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'Gene': ['HIST1H3G', 'HIST1H3G', 'HIST1H3G', 'TNFAIP8L1', 'OTOP2']}



Gene expression data shape after mapping: (19521, 37)

First 10 genes after mapping:
['A1BG', 'A1CF', 'A2BP1', 'A2LD1', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1']

Number of genes mapped from multiple probes: 12611
Top 5 genes with the most probes:
Gene
---       600
NF1        21
NFATC4     16
FMNL1      16
DMKN       16
Name: count, dtype: int64



Gene data shape after normalization: (19298, 37)


### Step 7: Data Normalization and Linking

In [8]:
# 1. First save the normalized gene expression data from the previous step
print("Saving normalized gene expression data...")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to: {out_gene_data_file}")

# 2. Extract the clinical data using the convert functions defined in step 2
print("\nExtracting clinical data...")
clinical_df = geo_select_clinical_features(
    clinical_data,  # First parameter is the dataframe
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data shape:", clinical_df.shape)
print("Clinical data preview:")
print(preview_df(clinical_df, n=5))

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to: {out_clinical_data_file}")

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# 4. Handle missing values
print("\nHandling missing values...")
linked_data_clean = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
is_biased, linked_data_clean = judge_and_remove_biased_features(linked_data_clean, trait)

# 6. Conduct final quality validation
note = "This GSE83500 dataset contains gene expression data from aortic wall of patients with ischemic heart disease, comparing MI patients with non-MI patients. Clinical data includes age, gender, and MI status."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Saving normalized gene expression data...


Normalized gene data saved to: ../../output/preprocess/Atherosclerosis/gene_data/GSE83500.csv

Extracting clinical data...
Clinical data shape: (3, 37)
Clinical data preview:
{'GSM2204583': [1.0, 69.0, 1.0], 'GSM2204584': [1.0, 56.0, 1.0], 'GSM2204585': [1.0, 56.0, 1.0], 'GSM2204586': [1.0, 53.0, 1.0], 'GSM2204587': [1.0, 58.0, 1.0], 'GSM2204588': [1.0, 70.0, 1.0], 'GSM2204589': [1.0, 50.0, 1.0], 'GSM2204590': [1.0, 61.0, 0.0], 'GSM2204591': [1.0, 63.0, 1.0], 'GSM2204592': [1.0, 56.0, 1.0], 'GSM2204593': [1.0, 65.0, 1.0], 'GSM2204594': [1.0, 58.0, 1.0], 'GSM2204595': [1.0, 81.0, 0.0], 'GSM2204596': [1.0, 68.0, 0.0], 'GSM2204597': [1.0, 62.0, 1.0], 'GSM2204598': [1.0, 64.0, 1.0], 'GSM2204599': [1.0, 50.0, 1.0], 'GSM2204600': [1.0, 81.0, 0.0], 'GSM2204601': [1.0, 78.0, 1.0], 'GSM2204602': [1.0, 56.0, 1.0], 'GSM2204603': [1.0, 52.0, 0.0], 'GSM2204604': [1.0, 55.0, 1.0], 'GSM2204605': [1.0, 48.0, 1.0], 'GSM2204606': [1.0, 49.0, 1.0], 'GSM2204607': [1.0, 55.0, 1.0], 'GSM2204608': [1.0, 64.0

Linked data shape after handling missing values: (37, 19301)

Checking for bias in dataset features...
Quartiles for 'Atherosclerosis':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Atherosclerosis' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 54.0
  50% (Median): 58.0
  75%: 64.0
Min: 48.0
Max: 81.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 5 occurrences. This represents 13.51% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset deemed not usable for associative studies. Linked data not saved.
